In [25]:
import logging

In [26]:
# Set up basic configuration for logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [27]:
def calculcate_and_upload_drainage(bounds, is_final):
    bounds_str = boundstr(bounds)    # String form of chunk bounds
    tile_id = xy_to_tile_id(bounds[0], bounds[3])    # tile_id in YYN/S_XXXE/W
    chunk_length_pixels = calc_chunk_length_pixels(bounds)   # Chunk length in pixels (as opposed to decimal degrees

    no_data_val = 255
    logging.info(f"Processing tile {tile_id} with bounds {bounds_str}")

    try: 
        # Dictionary of downloaded layers
        download_dict = {}
        layers = {}
    
        download_dict = {
            # f"{land_cover}_2000": f"{LC_uri}/composite/2000/raw/{tile_id}.tif",
            # f"{land_cover}_2005": f"{LC_uri}/composite/2005/raw/{tile_id}.tif",
            # f"{land_cover}_2010": f"{LC_uri}/composite/2010/raw/{tile_id}.tif",
            # f"{land_cover}_2015": f"{LC_uri}/composite/2015/raw/{tile_id}.tif"
            f"{land_cover}_2020": f"s3://gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_classes/2020/40000_pixels/20240205/{tile_id}__IPCC_classes_2020.tif",
            planted_forest_type_layer: f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/plantation_type/SDPTv2/20230911/{tile_id}_plantation_type_oilpalm_woodfiber_other.tif", # Originally from gfw-data-lake, so it's in 400x400 windows
            planted_forest_tree_crop_layer: f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/plantation_simpleType__planted_forest_tree_crop/SDPTv2/20230911/{tile_id}.tif",  # Originally from gfw-data-lake, so it's in 400x400 windows
            peat: f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/peatlands/processed/20230315/{tile_id}_peat_mask_processed.tif",
            dadap: f"s3://gfw2-data/climate/AFOLU_flux_model/organic_soils/inputs/processed/dadap_density/dadap_{tile_id}.tif",
            ecozone: f"s3://gfw2-data/fao_ecozones/v2000/raster/epsg-4326/10/40000/class/gdal-geotiff/{tile_id}.tif",   # Originally from gfw-data-lake, so it's in 400x400 windows 

        }
    
        # Checks whether tile exists at all. Doesn't try to download chunk if the tile doesn't exist.
        tile_exists = check_for_tile(download_dict, is_final)
    
        if tile_exists == 0:
            return
    
        # Note: If running in a local Dask cluster, prints to console may be duplicated. Doesn't happen with a Coiled cluster of the same size (1 worker).
        # Seems to be a problem with local Dask getting overwhelmed by so many futures being created and downloaded from s3. 
        futures = prepare_to_download_chunk(bounds, download_dict, no_data_val)
    
        # print(futures)
    
        if not is_final:
            print(f"Waiting for requests for data in chunk {bounds_str} in {tile_id}: {timestr()}")
        
        # Waits for requests to come back with data from S3
        for future in concurrent.futures.as_completed(futures):
            layer = futures[future]
            layers[layer] = future.result()
    
        print(layers)
        # print(layers[soil_c_2000].dtype)
        
        # Checks chunk for data. Skips the chunk if it has no data in it.
        # will update this line to the peat version 
        data_in_chunk = check_chunk_for_data(layers, f"{land_cover}_", bounds_str, tile_id, no_data_val, is_final)
    
        if data_in_chunk == 0:
            return
    
        
        ### Part 2: Create a separate dictionary for each chunk datatype so that they can be passed to Numba as separate arguments.
        ### Numba functions can accept (and return) dictionaries of arrays as long as each dictionary only has arrays of one data type (e.g., uint8, float32)
        ### Note: need to add new code if inputs with other data types are added
    
        # Initializes empty dictionaries for each type
        uint8_dict_layers = {}
        int16_dict_layers = {}
        float32_dict_layers = {}
        
        # Iterates through the downloaded chunk dictionary and distributes arrays to a separate dictionary for each data type
        for key, array in layers.items():
            if array.dtype == np.uint8:
                uint8_dict_layers[key] = array
            elif array.dtype == np.int16:
                int16_dict_layers[key] = array
            elif array.dtype == np.float32:
                float32_dict_layers[key] = array
            else:
                raise TypeError(f"{key} dtype not in list")
    
        # print(f"uint8 datasets: {uint8_dict_layers.keys()}")
        # print(f"int16 datasets: {int16_dict_layers.keys()}")
        # print(f"float32 datasets: {float32_dict_layers.keys()}")
        
        # Creates numba-compliant typed dict for each type of array
        typed_dict_uint8 = Dict.empty(
            key_type=types.unicode_type, 
            value_type=types.Array(types.uint8, 2, 'C')  # Assuming 2D arrays of uint8
        )
    
        typed_dict_int16 = Dict.empty(
            key_type=types.unicode_type, 
            value_type=types.Array(types.int16, 2, 'C')  # Assuming 2D arrays of uint8
        )
    
        typed_dict_float32 = Dict.empty(
            key_type=types.unicode_type, 
            value_type=types.Array(types.float32, 2, 'C')  # Assuming 2D arrays of float32
        )
    
        # Populates the numba-compliant typed dicts
        for key, array in uint8_dict_layers.items():
            typed_dict_uint8[key] = array
    
        for key, array in int16_dict_layers.items():
            typed_dict_int16[key] = array
    
        for key, array in float32_dict_layers.items():
            typed_dict_float32[key] = array
    
    
        ### Part 3: Creates drainage map 
       
        print(f"Creating drainage map in {bounds_str} in {tile_id}: {timestr()}")

        ### processing is executed here 
        out_dict_uint32, out_dict_float32 = process_soil_types(
            typed_dict_uint8, typed_dict_int16, typed_dict_float32 
        )
    
        out_dict_all_dtypes = {}
    
        # Transfers the dictionaries of numpy arrays for each data type to a new, Pythonic array
        for key, value in out_dict_uint32.items():
            out_dict_all_dtypes[key] = value
    
        for key, value in out_dict_float32.items():
            out_dict_all_dtypes[key] = value
    
        # Clear memory of unneeded arrays
        del out_dict_uint32
        del out_dict_float32
    
        
        ### Part 4: Save numpy as rasters and upload to s3
    
        # Adds metadata used for uploading outputs to s3 to the dictionary
        for key, value in out_dict_all_dtypes.items():
    
            data_type = value.dtype.name
            out_pattern = key[:-10]
            year = int(key[-4:])
    
            # Dictionary with metadata for each array
            #will need to aler the year - year part 
            out_dict_all_dtypes[key] = [value, data_type, out_pattern, f'{year-5}_{year}']
    
        save_and_upload_small_raster_set(bounds, chunk_length_pixels, tile_id, bounds_str, out_dict_all_dtypes, is_final)
        
        # Clear memory of unneeded arrays
        del out_dict_all_dtypes
    
        return f"Success for {bounds_str}: {timestr()}"

    except Exception as e:
            logging.error(f"Failed processing for {bounds_str}: {str(e)}")

In [28]:
@jit(nopython=True)

### this is meant to outline the values of the decision tree. these will be replaced by a dictionary of states and values for each dataset
### may add extraction to the decision tree

def process_soil_types(in_dict_uint8, in_dict_int16, in_dict_float32):

    # Separate dictionaries for output numpy arrays of each datatype, named by output type (e.g., AGC density, BGC removals) and year ([output_type}_[year]).
    # This is because a dictionary in a Numba function cannot have arrays with multiple data types, so each dictionary has to store only one data type,
    # just like inputs to the function.
    out_dict_uint32 = {}
    out_dict_float32 = {}

    peat = in_dict_uint8[peat]
    land_cover = in_dict_uint8[land_cover_2020]
    rows, cols = peat.shape
    print(rows, cols)
    
 
    soil = np.empty((rows, cols), dtype=np.object_)
    state = np.empty((rows, cols), dtype=np.object_)

    node = 0  # Initialize node for decision tracking
    
    for row in range(rows):
        for col in range(cols):        
            node = 0  # Initialize node for decision tracking
            if peat[row, col] == 1:
                soil[row, col] = 'organic'
                if (dadap[row, col] > 0): #or (grip[row, col] > 0) or (osm[row, col] > 0) or (congo[row, col] > 0):
                    node = accrete_node(node, 1)  # Drained due to water management
                    state[row, col] = 'drained'
                # elif planted_forest_type[row, col] == 'plantation':
                #     node = accrete_node(node, 2)  # Drained due to being plantation
                #     state[row, col] = 'drained'
                elif land_cover[row, col] == "settlement" or land_cover[row, col] == "cropland":
                    node = accrete_node(node, 3)  # Drained due to land cover type
                    state[row, col] = 'drained'
                else:
                    node = accrete_node(node, 4)  # Undrained
                    state[row, col] = 'undrained'
            else:
                soil[row, col] = 'inorganic'
                print('inorganic soil omitted from analysis')
                # No need to set a state for inorganic soils, continue to next pixel
                continue

    return soil, state

In [29]:
%%time

## Create LULUCF flux and carbon stock 2x2 deg rasters 

## Area to analyze
## chunk_params arguments: W, S, E, N, chunk size (degrees)
# chunk_params = [-180, -60, 180, 80, 2]  # entire world
# chunk_params = [-10, 40, 20, 70, 1]    # 30x30 deg (70N_010W), 900 chunks

# chunk_params = [-10, 60, 0, 70, 1]    # 10x10 deg (70N_010W), 100 chunks
# chunk_params = [-10, 65, -5, 70, 1]    # 5x5 deg (70N_010W), 25 chunks
# chunk_params = [-10, 68, -8, 70, 1]    # 2x2 deg (70N_010W), 4 chunks
# chunk_params = [-10, 69, -9, 70, 1]    # 1x1 deg (70N_010W), 1 chunk

# chunk_params = [10, 40, 20, 50, 2]    # 10x10 deg (50N_010E), 25 chunks
# chunk_params = [10, 40, 20, 50, 10]    # 10x10 deg (50N_010E), 1 chunk
# chunk_params = [10, 46, 14, 50, 2]   # 4x4 deg, 4 chunks
# chunk_params = [110, -10, 114, -6, 2]   # 4x4 deg, 4 chunks
# chunk_params = [10, 48, 12, 50, 1]   # 2x2 deg, 4 chunks
# chunk_params = [10, 49, 11, 50, 1]   # 1x1 deg, 1 chunk
# chunk_params = [10, 49, 11, 50, 0.5] # 1x1 deg, 4 chunks
# chunk_params = [10, 49.5, 10.5, 50, 0.25] # 0.5x0.5 deg, 4 chunks
# chunk_params = [10, 42, 11, 43, 0.5] # 1x1 deg, 4 chunks (some GLCLU code=254 for ocean and some land, so data should be output)
# chunk_params = [10, 49.75, 10.25, 50, 0.25] # 0.25x0.25 deg, 1 chunk (has data, no fire)
#chunk_params = [15, 41.75, 15.25, 42, 0.25] # 0.25x0.25 deg, 1 chunk (has data with fire)

# # Range of no-data cases for testing
# chunk_params = [20, 69.75, 20.25, 70, 0.25] # 0.25x0.25 deg, 1 chunk (tile exists for GLCLU but not all other inputs, e.g., fire)
# chunk_params = [110, -10, 120, 0, 2]    # 10x10 deg (00N_110E), 25 chunks (all chunks have land and should be output)
# chunk_params = [110, -20, 120, -10, 2]    # 10x10 deg (00N_110E), 25 chunks (all chunks have land and should be output)
# chunk_params = [0, 79.75, 0.25, 80, 0.25] # 0.25x0.25 deg, 1 chunk (no 80N_000E tile-- no data)
# chunk_params = [112, -12, 116, -8, 2]   # 2x2 deg, 1 chunk (bottom of Java, has data but mostly ocean)
# chunk_params = [10.875, 41.75, 11, 42, 0.25] # 0.25x0.25 deg, 1 chunk (entirely GLCLU code=255 for ocean, so no actual data-- nothing should be be output)
# chunk_params = [-10, 21.75, -9.75, 22, 0.25] # 0.25x0.25 deg, 1 chunk (has data but entirely desert (fully GLCLU code=0))
# chunk_params = [10, 49.75, 10.25, 50, 0.25] # 0.25x0.25 deg, 1 chunk (has data)

chunk_params = [116, -6, 118, -4, 2] # 1 chunk, has data

# Makes list of chunks to analyze
chunks = get_chunk_bounds(chunk_params)  
print("Processing", len(chunks), "chunks")
# print(chunks)

# Determines if the output file names for final versions of outputs should be used
is_final = False
if len(chunks) > 90:
    is_final = True
    print("Running as final model.")

# Creates list of tasks to run (1 task = 1 chunk for all years)
delayed_result = [dask.delayed(calculcate_and_upload_drainage)(chunk, is_final) for chunk in chunks]

# Actually runs analysis
results = dask.compute(*delayed_result)
results

2024-05-11 13:00:57,212 - INFO - Processing tile 00N_110E with bounds 116_-6_118_-4


Processing 1 chunks


2024-05-11 13:00:57,605 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2024-05-11 13:00:57,615 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2024-05-11 13:00:57,643 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2024-05-11 13:00:57,652 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2024-05-11 13:00:57,653 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2024-05-11 13:00:57,654 - INFO - Found credentials in shared credentials file: ~/.aws/credentials


Tile id ses_2020 exists. Proceeding.
Requesting data in chunk 116_-6_118_-4 in 00N_110E: 20240511_13_00_57


2024-05-11 13:03:11,868 - ERROR - Failed processing for 116_-6_118_-4: ecozone dtype not in list


Waiting for requests for data in chunk 116_-6_118_-4 in 00N_110E: 20240511_13_03_11
{'peat': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), 'land_cover_2020': array([[6, 6, 6, ..., 0, 6, 6],
       [6, 6, 6, ..., 0, 6, 6],
       [6, 6, 6, ..., 0, 6, 6],
       ...,
       [0, 0, 0, ..., 6, 6, 6],
       [6, 6, 6, ..., 6, 6, 6],
       [6, 6, 6, ..., 6, 6, 6]], dtype=uint8), 'planted_forest_type': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), 'planted_forest_tree_crop': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [

(None,)